In [1]:
import pandas as pd

# Carregar o arquivo enviado pelo usuário
file_path = '/content/base_dados_teste_analista.csv'
data = pd.read_csv(file_path)

# Exibir as primeiras linhas do dataset para entender sua estrutura
data.head(), data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Data da Venda        1000 non-null   object 
 1   Produto              1000 non-null   object 
 2   Categoria            1000 non-null   object 
 3   Quantidade Vendida   1000 non-null   int64  
 4   Receita              1000 non-null   float64
 5   Custo                1000 non-null   float64
 6   Localidade           1000 non-null   object 
 7   Vendedor             1000 non-null   object 
 8   Lucro                1000 non-null   float64
 9   Margem de Lucro (%)  1000 non-null   float64
dtypes: float64(4), int64(1), object(5)
memory usage: 78.2+ KB


(  Data da Venda    Produto   Categoria  Quantidade Vendida  Receita   Custo  \
 0    2023-04-13  Produto B      Roupas                  11   570.78  379.87   
 1    2023-12-15  Produto D   Alimentos                   3   703.09  287.86   
 2    2023-09-28  Produto B  Brinquedos                   1   936.65  407.03   
 3    2023-04-17  Produto D  Brinquedos                   6   579.03  495.75   
 4    2023-03-13  Produto A   Alimentos                  18   622.84  140.29   
 
        Localidade Vendedor   Lucro  Margem de Lucro (%)  
 0       São Paulo  Mariana  190.91                33.45  
 1  Belo Horizonte  Mariana  415.23                59.06  
 2       São Paulo     João  529.62                56.54  
 3        Curitiba   Carlos   83.28                14.38  
 4       São Paulo     João  482.55                77.48  ,
 None)

In [3]:
# Converter a coluna 'Data da Venda' para datetime
data['Data da Venda'] = pd.to_datetime(data['Data da Venda'])

# Criar uma base agregada por data (soma de vendas por dia)
aggregated_data = data.groupby('Data da Venda')['Quantidade Vendida'].sum().reset_index()

# Exibir as primeiras linhas dos dados agregados
aggregated_data.head()


,Data da Venda,Quantidade Vendida
0,2023-01-01,73
1,2023-01-02,34
2,2023-01-03,18
3,2023-01-04,33
4,2023-01-05,56


In [8]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Configurar o modelo de previsão com suavização exponencial
model = ExponentialSmoothing(
    aggregated_data['Quantidade Vendida'],
    seasonal='add',
    seasonal_periods=7,  # Supondo sazonalidade semanal
    trend='add',
    initialization_method='estimated'
)

# Treinar o modelo
fitted_model = model.fit()

# Fazer previsões para o próximo ano (365 dias)
future_predictions = fitted_model.forecast(steps=365)

# Criar um DataFrame com as previsões
future_dates = pd.date_range(start=aggregated_data['Data da Venda'].iloc[-1] + pd.Timedelta(days=1), periods=365)
predictions_df = pd.DataFrame({
    'Data da Venda': future_dates,
    'Quantidade Prevista': future_predictions
})

# Concatenar os dados reais e previstos
result_data = pd.concat([aggregated_data, predictions_df.set_index('Data da Venda')], axis=0)

# Exibir as últimas linhas para verificar as previsões
result_data.tail(365)


,Data da Venda,Quantidade Vendida,Quantidade Prevista
2024-01-01 00:00:00,NaT,NaN,29.370884
2024-01-02 00:00:00,NaT,NaN,26.806728
2024-01-03 00:00:00,NaT,NaN,28.078762
2024-01-04 00:00:00,NaT,NaN,31.912985
2024-01-05 00:00:00,NaT,NaN,31.106269
...,...,...,...
2024-12-26 00:00:00,NaT,NaN,20.946364
2024-12-27 00:00:00,NaT,NaN,20.139648
2024-12-28 00:00:00,NaT,NaN,17.054533
2024-12-29 00:00:00,NaT,NaN,17.980384


In [10]:
# Criar DataFrame com as previsões
future_dates = pd.date_range(start=aggregated_data['Data da Venda'].iloc[-1] + pd.Timedelta(days=1), periods=365)
predictions_df = pd.DataFrame({
    'Data Futura': future_dates,
    'Valor Previsto': future_predictions
})

# Exibir as últimas linhas para verificar as previsões
predictions_df.tail(10)


,Data Futura,Valor Previsto
691,2024-12-21,17.269565
692,2024-12-22,18.195415
693,2024-12-23,18.404263
694,2024-12-24,15.840107
695,2024-12-25,17.112141
696,2024-12-26,20.946364
697,2024-12-27,20.139648
698,2024-12-28,17.054533
699,2024-12-29,17.980384
700,2024-12-30,18.189231


In [13]:
# prompt: armazene o resultado em um arquivo csv, chamando predicao

# Salvar os resultados em um arquivo CSV chamado 'predicao.csv'
predictions_df.to_csv('previsao.csv', index=False)